In [1]:
import sqlite3
from sqlite3 import Error
from sqlite3 import IntegrityError
from ipyfilechooser import FileChooser
import pandas as pd
import io
import requests
import numpy
from scipy import stats
import math
from scipy.stats import ttest_ind
import ipywidgets as widgets
import tqdm

In [2]:
starting_directory = '/Volumes'
baseline_chooser = FileChooser(starting_directory)
display(baseline_chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [3]:
tap_chooser=FileChooser(starting_directory)
display(tap_chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [4]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'Neuron_Genes_Screen']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

Select(description='Screen:', options=('PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'N…

In [5]:
Screen=screen_chooser.value
folder_path=baseline_chooser.selected_path
print(folder_path)

/Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022


In [6]:
# Read the baseline file
baseline_output = pd.read_csv(baseline_chooser.selected, index_col=0).drop(columns=['index'])

print(f"\nShape of the baseline .csv file: {baseline_output.shape}")

# Print the first five rows of the file
baseline_output.head()


Shape of the baseline .csv file: (30487, 21)


,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,...,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,dataset,Gene,Allele
0,490.016,14,12,0.0823,0.1195,0.25,0.1078,1.0908,0.142641,6.1,...,31.1,25.0,0.0066,11.893,B0811ab,20220815,PD_Screen,N2,N2,N2
1,490.056,14,12,0.0736,0.1024,0.25,0.1059,1.0880,0.140150,5.3,...,30.9,24.7,0.0064,11.896,B0811ab,20220815,PD_Screen,N2,N2,N2
2,490.103,14,12,0.0784,0.1024,0.25,0.1050,1.0914,0.138935,5.2,...,31.0,24.6,0.0057,11.898,B0811ab,20220815,PD_Screen,N2,N2,N2
3,490.144,14,12,0.0970,0.1118,0.25,0.1054,1.0935,0.140575,5.8,...,30.6,24.5,0.0103,11.901,B0811ab,20220815,PD_Screen,N2,N2,N2
4,490.186,14,12,0.0994,0.1197,0.25,0.1110,1.1026,0.146894,5.6,...,31.4,24.4,0.0097,11.904,B0811ab,20220815,PD_Screen,N2,N2,N2


In [7]:
# Read the tap file
tap_output = pd.read_csv(tap_chooser.selected, index_col=0)

print(f"\nShape of the tap .csv file: {tap_output.shape}")

# Print the first five rows of the file
tap_output.head()


Shape of the tap .csv file: (395, 13)


,time,dura,dist,prob,speed,plate,Date,Plate_id,Screen,taps,dataset,Gene,Allele
0,599.985,2.83,0.696,0.928571,0.245936,1,20220815,B0811ab,PD_Screen,1.0,N2,N2,N2
1,609.993,2.98,0.746,0.857143,0.250336,1,20220815,B0811ab,PD_Screen,2.0,N2,N2,N2
2,619.699,1.97,0.536,0.800000,0.272081,1,20220815,B0811ab,PD_Screen,3.0,N2,N2,N2
3,629.956,2.57,0.686,0.900000,0.266926,1,20220815,B0811ab,PD_Screen,4.0,N2,N2,N2
4,639.957,1.34,0.383,0.909091,0.285821,1,20220815,B0811ab,PD_Screen,5.0,N2,N2,N2


In [8]:
# Dataframe for first tap
PD_first_tap = (
    tap_output[(tap_output.taps==1)]
    .reset_index().drop(columns="index")
    .rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
)

PD_first_tap

,time,init_dura,dist,init_prob,init_speed,plate,Date,Plate_id,Screen,taps,dataset,Gene,Allele
0,599.985,2.83,0.696,0.928571,0.245936,1,20220815,B0811ab,PD_Screen,1.0,N2,N2,N2
1,599.991,2.14,0.524,0.937500,0.244860,2,20220815,A0811aa,PD_Screen,1.0,N2,N2,N2
2,599.981,2.96,0.783,1.000000,0.264527,3,20220815,A0811ad,PD_Screen,1.0,N2,N2,N2
3,599.962,2.38,0.501,0.913043,0.210504,4,20220815,B0811ae,PD_Screen,1.0,N2,N2,N2
4,599.972,2.52,0.622,0.863636,0.246825,5,20220815,C0811ac,PD_Screen,1.0,N2,N2,N2
5,599.958,2.07,0.820,0.800000,0.396135,1,20220815,A0811bc,PD_Screen,1.0,hipr-1_ok1081,hipr-1,ok1081
6,599.990,2.50,0.549,0.578947,0.219600,2,20220815,C0811bb,PD_Screen,1.0,hipr-1_ok1081,hipr-1,ok1081
7,599.978,1.63,0.429,1.000000,0.263190,3,20220815,B0811bd,PD_Screen,1.0,hipr-1_ok1081,hipr-1,ok1081
8,599.982,3.24,0.874,0.900000,0.269753,4,20220815,C0811be,PD_Screen,1.0,hipr-1_ok1081,hipr-1,ok1081
9,599.988,2.28,0.401,0.700000,0.175877,1,20220815,B0811cc,PD_Screen,1.0,hipr-1_tm10120,hipr-1,tm10120


In [9]:
# Dataframe for recovery taps
PD_recov_taps = (
    tap_output[(tap_output.taps==31)]
    .reset_index().drop(columns="index")
    .rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
)

PD_recov_taps

,time,recov_dura,dist,recov_prob,recov_speed,plate,Date,Plate_id,Screen,taps,dataset,Gene,Allele
0,1189.987,2.61,0.464,0.545455,0.177778,1,20220815,B0811ab,PD_Screen,31.0,N2,N2,N2
1,1189.979,1.97,0.445,0.692308,0.225888,2,20220815,A0811aa,PD_Screen,31.0,N2,N2,N2
2,1189.962,1.79,0.610,0.882353,0.340782,3,20220815,A0811ad,PD_Screen,31.0,N2,N2,N2
3,1189.978,1.70,0.440,0.761905,0.258824,4,20220815,B0811ae,PD_Screen,31.0,N2,N2,N2
4,1189.966,2.01,0.561,0.607143,0.279104,5,20220815,C0811ac,PD_Screen,31.0,N2,N2,N2
5,1189.957,1.85,0.599,1.000000,0.323784,1,20220815,A0811bc,PD_Screen,31.0,hipr-1_ok1081,hipr-1,ok1081
6,1189.983,2.62,0.832,0.750000,0.317557,2,20220815,C0811bb,PD_Screen,31.0,hipr-1_ok1081,hipr-1,ok1081
7,1189.973,1.38,0.318,1.000000,0.230435,3,20220815,B0811bd,PD_Screen,31.0,hipr-1_ok1081,hipr-1,ok1081
8,1189.973,1.57,0.507,0.888889,0.322930,4,20220815,C0811be,PD_Screen,31.0,hipr-1_ok1081,hipr-1,ok1081
9,1189.989,2.26,0.489,0.880000,0.216372,1,20220815,B0811cc,PD_Screen,31.0,hipr-1_tm10120,hipr-1,tm10120


In [10]:
# Dataframe for last three taps
PD_final_taps = (
    tap_output[((tap_output.taps >= 28) & (tap_output.taps <= 30))]
    .groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"])
    .mean()
    .reset_index()
    .rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")
)

PD_final_taps

,dataset,Date,Plate_id,Screen,Gene,Allele,plate,time,final_dura,dist,final_prob,final_speed,taps
0,N2,20220815,A0811aa,PD_Screen,N2,N2,2,879.973000,0.990000,0.233333,0.302832,0.221688,29.0
1,N2,20220815,A0811ad,PD_Screen,N2,N2,3,879.989333,0.600000,0.124000,0.319444,0.202159,29.0
2,N2,20220815,B0811ab,PD_Screen,N2,N2,1,879.936333,1.233333,0.206333,0.355556,0.162658,29.0
3,N2,20220815,B0811ae,PD_Screen,N2,N2,4,879.990000,0.846667,0.196000,0.314762,0.237710,29.0
4,N2,20220815,C0811ac,PD_Screen,N2,N2,5,879.968333,1.346667,0.341000,0.381481,0.248545,29.0
5,hipr-1_ok1081,20220815,A0811bc,PD_Screen,hipr-1,ok1081,1,879.964000,2.180000,0.381000,0.500000,0.174771,29.0
6,hipr-1_ok1081,20220815,B0811bd,PD_Screen,hipr-1,ok1081,3,879.969000,0.990000,0.193667,0.606061,0.185183,29.0
7,hipr-1_ok1081,20220815,C0811bb,PD_Screen,hipr-1,ok1081,2,879.986667,1.143333,0.315000,0.508578,0.264993,29.0
8,hipr-1_ok1081,20220815,C0811be,PD_Screen,hipr-1,ok1081,4,879.978333,1.063333,0.299000,0.578704,0.280136,29.0
9,hipr-1_tm10120,20220815,A0811cb,PD_Screen,hipr-1,tm10120,4,879.981000,0.976667,0.201333,0.734271,0.203894,29.0


In [11]:
# Dataframe to analyse habituation behaviour after merging first tap and final taps

PD_habit_levels = pd.merge(
    PD_first_tap, 
    PD_final_taps, 
    on =['dataset', 'plate', "Plate_id", "Screen", "Gene", "Allele", "Date"], how ='left'
).drop(columns=['time_x','time_y','dist_x','dist_y', 'taps_x', 'taps_y']).dropna()

PD_habit_levels['habit_dura'] = PD_habit_levels['init_dura'] - PD_habit_levels['final_dura']

PD_habit_levels['habit_prob'] = PD_habit_levels['init_prob'] - PD_habit_levels['final_prob']

PD_habit_levels['habit_speed'] = PD_habit_levels['init_speed'] - PD_habit_levels['final_speed']


PD_habit_levels

,init_dura,init_prob,init_speed,plate,Date,Plate_id,Screen,dataset,Gene,Allele,final_dura,final_prob,final_speed,habit_dura,habit_prob,habit_speed
0,2.83,0.928571,0.245936,1,20220815,B0811ab,PD_Screen,N2,N2,N2,1.233333,0.355556,0.162658,1.596667,0.573016,0.083278
1,2.14,0.937500,0.244860,2,20220815,A0811aa,PD_Screen,N2,N2,N2,0.990000,0.302832,0.221688,1.150000,0.634668,0.023172
2,2.96,1.000000,0.264527,3,20220815,A0811ad,PD_Screen,N2,N2,N2,0.600000,0.319444,0.202159,2.360000,0.680556,0.062368
3,2.38,0.913043,0.210504,4,20220815,B0811ae,PD_Screen,N2,N2,N2,0.846667,0.314762,0.237710,1.533333,0.598282,-0.027205
4,2.52,0.863636,0.246825,5,20220815,C0811ac,PD_Screen,N2,N2,N2,1.346667,0.381481,0.248545,1.173333,0.482155,-0.001719
5,2.07,0.800000,0.396135,1,20220815,A0811bc,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,2.180000,0.500000,0.174771,-0.110000,0.300000,0.221365
6,2.50,0.578947,0.219600,2,20220815,C0811bb,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,1.143333,0.508578,0.264993,1.356667,0.070369,-0.045393
7,1.63,1.000000,0.263190,3,20220815,B0811bd,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,0.990000,0.606061,0.185183,0.640000,0.393939,0.078008
8,3.24,0.900000,0.269753,4,20220815,C0811be,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,1.063333,0.578704,0.280136,2.176667,0.321296,-0.010383
9,2.28,0.700000,0.175877,1,20220815,B0811cc,PD_Screen,hipr-1_tm10120,hipr-1,tm10120,1.306667,0.666638,0.187877,0.973333,0.033362,-0.011999


In [12]:
# Continue to analyse habituation behaviour after merging with recovery taps

if PD_recov_taps.empty:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='outer')
else:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')

if Screen not in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    PD_habituation = PD_habituation.dropna() 

PD_habituation['recovery_dura']=(PD_habituation.recov_dura-PD_habituation.init_dura)/PD_habituation.init_dura*100

PD_habituation['recovery_prob']=(PD_habituation.recov_prob-PD_habituation.init_prob)/PD_habituation.init_prob*100

PD_habituation['recovery_speed']=(PD_habituation.recov_speed-PD_habituation.init_speed)/PD_habituation.init_speed*100

PD_habituation['memory_retention_dura']=(PD_habituation.recov_dura-PD_habituation.final_dura)

PD_habituation['memory_retention_prob']=(PD_habituation.recov_prob-PD_habituation.final_prob)

PD_habituation['memory_retention_speed']=(PD_habituation.recov_speed-PD_habituation.final_speed)


# Rename `PD_habituation` to `tap_data` based on the condition below
if Screen in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    tap_data=PD_habituation.dropna(subset = ['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed'])
else:
    tap_data=PD_habituation.dropna() 


# Display final dataframe
tap_data


,init_dura,init_prob,init_speed,plate,Date,Plate_id,Screen,dataset,Gene,Allele,...,dist,recov_prob,recov_speed,taps,recovery_dura,recovery_prob,recovery_speed,memory_retention_dura,memory_retention_prob,memory_retention_speed
0,2.83,0.928571,0.245936,1,20220815,B0811ab,PD_Screen,N2,N2,N2,...,0.464,0.545455,0.177778,31.0,-7.773852,-41.258741,-27.713921,1.376667,0.189899,0.015119
1,2.14,0.937500,0.244860,2,20220815,A0811aa,PD_Screen,N2,N2,N2,...,0.445,0.692308,0.225888,31.0,-7.943925,-26.153846,-7.747898,0.980000,0.389475,0.004200
2,2.96,1.000000,0.264527,3,20220815,A0811ad,PD_Screen,N2,N2,N2,...,0.610,0.882353,0.340782,31.0,-39.527027,-11.764706,28.826958,1.190000,0.562908,0.138623
3,2.38,0.913043,0.210504,4,20220815,B0811ae,PD_Screen,N2,N2,N2,...,0.440,0.761905,0.258824,31.0,-28.571429,-16.553288,22.954092,0.853333,0.447143,0.021114
4,2.52,0.863636,0.246825,5,20220815,C0811ac,PD_Screen,N2,N2,N2,...,0.561,0.607143,0.279104,31.0,-20.238095,-29.699248,13.077698,0.663333,0.225661,0.030560
5,2.07,0.800000,0.396135,1,20220815,A0811bc,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,...,0.599,1.000000,0.323784,31.0,-10.628019,25.000000,-18.264338,-0.330000,0.500000,0.149013
6,2.50,0.578947,0.219600,2,20220815,C0811bb,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,...,0.832,0.750000,0.317557,31.0,4.800000,29.545455,44.607127,1.476667,0.241422,0.052565
7,1.63,1.000000,0.263190,3,20220815,B0811bd,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,...,0.318,1.000000,0.230435,31.0,-15.337423,0.000000,-12.445525,0.390000,0.393939,0.045252
8,3.24,0.900000,0.269753,4,20220815,C0811be,PD_Screen,hipr-1_ok1081,hipr-1,ok1081,...,0.507,0.888889,0.322930,31.0,-51.543210,-1.234568,19.713157,0.506667,0.310185,0.042794
9,2.28,0.700000,0.175877,1,20220815,B0811cc,PD_Screen,hipr-1_tm10120,hipr-1,tm10120,...,0.489,0.880000,0.216372,31.0,-0.877193,25.714286,23.024298,0.953333,0.213362,0.028495


In [167]:
# # tap_url = 'https://osf.io/du9bj/files/osfstorage/650a2f9f1e76a4230e8a99a5?raw=true'
# tap_url='https://github.com/MyYummyPancake/NRSC510B/blob/main/tap_output.csv?raw=true'
# # s=requests.get(tap_url).content
# # tap_output=pd.read_csv(io.StringIO(s.decode('utf-8')))
# tap_output=pd.read_csv(tap_url, on_bad_lines='skip', index_col=0)
# print(tap_output)

In [168]:
# print(tap_output['Gene'].unique())
# print(len(tap_output['Gene'].unique()))
# print(baseline_output['Gene'].unique())
# print(len(baseline_output['Gene'].unique()))

In [169]:
# for x in tap_output['Gene'].unique():
#     tap_output_gene=tap_output[tap_output['Gene']==x]
#     gene_tap_data=tap_output[tap_output['Date'].isin(tap_output_gene['Date'].unique())]
#     gene_tap_data_final=gene_tap_data[gene_tap_data['Gene'].isin(['N2', x])]
#     gene_tap_data_final['taps']=gene_tap_data_final['taps'].astype(int)


# for x in baseline_output['Gene'].unique():
#     baseline_output_gene=baseline_output[baseline_output['Gene']==x]
#     gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
#     gene_baseline_data_final=gene_baseline_data[gene_baseline_data['Gene'].isin(['N2', x])]
#     for a,b in zip(list_baseline_metrics, list_baseline_Tstats):
#         TTest_Allele(x, a, baseline_output, b)


# Run Statistics (T-Test and sample-mean distance) on Data

In [ ]:
print(tap_data.columns)

Index(['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed', 'time',
       'recov_dura', 'dist', 'recov_prob', 'recov_speed', 'taps',
       'recovery_dura', 'recovery_prob', 'recovery_speed',
       'memory_retention_dura', 'memory_retention_prob',
       'memory_retention_speed'],
      dtype='object')


In [13]:
tap_data.head()

,init_dura,init_prob,init_speed,plate,Date,Plate_id,Screen,dataset,Gene,Allele,...,dist,recov_prob,recov_speed,taps,recovery_dura,recovery_prob,recovery_speed,memory_retention_dura,memory_retention_prob,memory_retention_speed
0,2.83,0.928571,0.245936,1,20220815,B0811ab,PD_Screen,N2,N2,N2,...,0.464,0.545455,0.177778,31.0,-7.773852,-41.258741,-27.713921,1.376667,0.189899,0.015119
1,2.14,0.937500,0.244860,2,20220815,A0811aa,PD_Screen,N2,N2,N2,...,0.445,0.692308,0.225888,31.0,-7.943925,-26.153846,-7.747898,0.980000,0.389475,0.004200
2,2.96,1.000000,0.264527,3,20220815,A0811ad,PD_Screen,N2,N2,N2,...,0.610,0.882353,0.340782,31.0,-39.527027,-11.764706,28.826958,1.190000,0.562908,0.138623
3,2.38,0.913043,0.210504,4,20220815,B0811ae,PD_Screen,N2,N2,N2,...,0.440,0.761905,0.258824,31.0,-28.571429,-16.553288,22.954092,0.853333,0.447143,0.021114
4,2.52,0.863636,0.246825,5,20220815,C0811ac,PD_Screen,N2,N2,N2,...,0.561,0.607143,0.279104,31.0,-20.238095,-29.699248,13.077698,0.663333,0.225661,0.030560


In [14]:
def get_output_byplate(output, baseline, allele):
    """
    Aggregates data by 'Plate_id','Date','Screen','dataset','Gene','Allele'

    Parameters:
        output (pd.DataFrame): Input DataFrame (either baseline_output or tap_data)
        baseline (boolean): whether data is baseline (True) or tap response (False)
        allele (boolean): group by allele (True) or group by gene (False)

    Returns:
        A DataFrame with plate-level averages
    """

    # columns to delete if baseline = True
    drop_col_base=['Plate_id','n','Number','Time','Screen','Date','Allele']

    # columns to delete if baseline = False
    drop_col_taps=['Plate_id','Screen','Date','Allele','dist','plate','time',
                   'taps','recov_dura','recov_prob','recov_speed']
    
    drop_col = drop_col_base if baseline else drop_col_taps

    drop_col.append('Gene') if allele else drop_col.append('dataset')
     
    output_byplate = output.groupby(
        by=['Plate_id','Date','Screen','dataset','Gene','Allele'],
        as_index=False).mean().drop(columns=drop_col)
    
    return output_byplate

##### `baseline` = True, `allele` = False

In [ ]:
baseline_output_byplate=get_output_byplate(baseline_output, baseline= True, allele=False)

print(f"Shape: {baseline_output_byplate.shape}")

baseline_output_byplate

Shape: (13, 13)


,Gene,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
0,N2,0.033536,0.032413,0.030834,0.116511,1.231836,0.169018,1.909915,0.261825,47.466667,29.442805,0.005735,8.756541
1,N2,0.059080,0.066365,0.203949,0.111072,1.194860,0.153823,3.502214,0.261859,49.367271,28.567150,0.008075,4.712773
2,hipr-1,0.111014,0.130682,0.426766,0.104552,1.100872,0.136805,10.201907,0.406782,63.183664,35.107551,0.016845,21.158225
3,hipr-1,0.027200,0.028163,0.008460,0.096320,0.937245,0.108482,2.342027,0.279167,48.979768,27.025200,0.004914,9.265980
4,N2,0.084010,0.103804,0.275993,0.108908,1.085004,0.142790,5.541694,0.272735,44.438571,28.031377,0.011272,8.938643
5,N2,0.050719,0.067794,0.152422,0.118605,1.110851,0.152327,3.539500,0.286462,49.287889,29.404889,0.007595,8.949534
6,hipr-1,0.049595,0.062754,0.167714,0.105314,0.944760,0.115941,4.970495,0.313570,47.140811,30.059910,0.008780,19.612836
7,hipr-1,0.020141,0.022212,0.007218,0.107844,0.884149,0.114715,1.997914,0.287647,39.681604,25.349626,0.005358,3.336735
8,N2,0.078540,0.084766,0.352588,0.114520,1.073642,0.146334,5.941411,0.285203,47.624689,28.439170,0.012888,11.070168
9,hipr-1,0.034934,0.040831,0.046652,0.110155,1.001404,0.132125,2.653416,0.281352,49.208320,27.844132,0.008648,10.239627


##### `baseline` = False, `allele` = False

In [ ]:
tap_data_byplate=get_output_byplate(tap_data, baseline=False, allele=False)

print(f"Shape: {tap_data_byplate.shape}")

tap_data_byplate

Shape: (13, 16)


,Gene,init_dura,init_prob,init_speed,final_dura,final_prob,final_speed,habit_dura,habit_prob,habit_speed,recovery_dura,recovery_prob,recovery_speed,memory_retention_dura,memory_retention_prob,memory_retention_speed
0,N2,2.14,0.937500,0.244860,0.990000,0.302832,0.221688,1.150000,0.634668,0.023172,-7.943925,-26.153846,-7.747898,0.980000,0.389475,0.004200
1,N2,2.96,1.000000,0.264527,0.600000,0.319444,0.202159,2.360000,0.680556,0.062368,-39.527027,-11.764706,28.826958,1.190000,0.562908,0.138623
2,hipr-1,2.07,0.800000,0.396135,2.180000,0.500000,0.174771,-0.110000,0.300000,0.221365,-10.628019,25.000000,-18.264338,-0.330000,0.500000,0.149013
3,hipr-1,2.53,0.826087,0.204348,0.976667,0.734271,0.203894,1.553333,0.091816,0.000454,-4.743083,21.052632,3.354816,1.433333,0.265729,0.007310
4,N2,2.83,0.928571,0.245936,1.233333,0.355556,0.162658,1.596667,0.573016,0.083278,-7.773852,-41.258741,-27.713921,1.376667,0.189899,0.015119
5,N2,2.38,0.913043,0.210504,0.846667,0.314762,0.237710,1.533333,0.598282,-0.027205,-28.571429,-16.553288,22.954092,0.853333,0.447143,0.021114
6,hipr-1,1.63,1.000000,0.263190,0.990000,0.606061,0.185183,0.640000,0.393939,0.078008,-15.337423,0.000000,-12.445525,0.390000,0.393939,0.045252
7,hipr-1,2.28,0.700000,0.175877,1.306667,0.666638,0.187877,0.973333,0.033362,-0.011999,-0.877193,25.714286,23.024298,0.953333,0.213362,0.028495
8,N2,2.52,0.863636,0.246825,1.346667,0.381481,0.248545,1.173333,0.482155,-0.001719,-20.238095,-29.699248,13.077698,0.663333,0.225661,0.030560
9,hipr-1,2.50,0.578947,0.219600,1.143333,0.508578,0.264993,1.356667,0.070369,-0.045393,4.800000,29.545455,44.607127,1.476667,0.241422,0.052565


##### `baseline` = True, `allele` = True

In [23]:
baseline_output_allele_byplate = get_output_byplate(baseline_output,baseline=True, allele=True)

print(f"Shape: {baseline_output_allele_byplate.shape}")

baseline_output_allele_byplate

Shape: (13, 13)


,dataset,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
0,N2,0.033536,0.032413,0.030834,0.116511,1.231836,0.169018,1.909915,0.261825,47.466667,29.442805,0.005735,8.756541
1,N2,0.059080,0.066365,0.203949,0.111072,1.194860,0.153823,3.502214,0.261859,49.367271,28.567150,0.008075,4.712773
2,hipr-1_ok1081,0.111014,0.130682,0.426766,0.104552,1.100872,0.136805,10.201907,0.406782,63.183664,35.107551,0.016845,21.158225
3,hipr-1_tm10120,0.027200,0.028163,0.008460,0.096320,0.937245,0.108482,2.342027,0.279167,48.979768,27.025200,0.004914,9.265980
4,N2,0.084010,0.103804,0.275993,0.108908,1.085004,0.142790,5.541694,0.272735,44.438571,28.031377,0.011272,8.938643
5,N2,0.050719,0.067794,0.152422,0.118605,1.110851,0.152327,3.539500,0.286462,49.287889,29.404889,0.007595,8.949534
6,hipr-1_ok1081,0.049595,0.062754,0.167714,0.105314,0.944760,0.115941,4.970495,0.313570,47.140811,30.059910,0.008780,19.612836
7,hipr-1_tm10120,0.020141,0.022212,0.007218,0.107844,0.884149,0.114715,1.997914,0.287647,39.681604,25.349626,0.005358,3.336735
8,N2,0.078540,0.084766,0.352588,0.114520,1.073642,0.146334,5.941411,0.285203,47.624689,28.439170,0.012888,11.070168
9,hipr-1_ok1081,0.034934,0.040831,0.046652,0.110155,1.001404,0.132125,2.653416,0.281352,49.208320,27.844132,0.008648,10.239627


##### `baseline` = False, `allele` = True

In [24]:
tap_data_allele_byplate = get_output_byplate(tap_data, baseline=False, allele=True)

print(f"Shape: {tap_data_allele_byplate.shape}")

tap_data_allele_byplate

Shape: (13, 16)


,dataset,init_dura,init_prob,init_speed,final_dura,final_prob,final_speed,habit_dura,habit_prob,habit_speed,recovery_dura,recovery_prob,recovery_speed,memory_retention_dura,memory_retention_prob,memory_retention_speed
0,N2,2.14,0.937500,0.244860,0.990000,0.302832,0.221688,1.150000,0.634668,0.023172,-7.943925,-26.153846,-7.747898,0.980000,0.389475,0.004200
1,N2,2.96,1.000000,0.264527,0.600000,0.319444,0.202159,2.360000,0.680556,0.062368,-39.527027,-11.764706,28.826958,1.190000,0.562908,0.138623
2,hipr-1_ok1081,2.07,0.800000,0.396135,2.180000,0.500000,0.174771,-0.110000,0.300000,0.221365,-10.628019,25.000000,-18.264338,-0.330000,0.500000,0.149013
3,hipr-1_tm10120,2.53,0.826087,0.204348,0.976667,0.734271,0.203894,1.553333,0.091816,0.000454,-4.743083,21.052632,3.354816,1.433333,0.265729,0.007310
4,N2,2.83,0.928571,0.245936,1.233333,0.355556,0.162658,1.596667,0.573016,0.083278,-7.773852,-41.258741,-27.713921,1.376667,0.189899,0.015119
5,N2,2.38,0.913043,0.210504,0.846667,0.314762,0.237710,1.533333,0.598282,-0.027205,-28.571429,-16.553288,22.954092,0.853333,0.447143,0.021114
6,hipr-1_ok1081,1.63,1.000000,0.263190,0.990000,0.606061,0.185183,0.640000,0.393939,0.078008,-15.337423,0.000000,-12.445525,0.390000,0.393939,0.045252
7,hipr-1_tm10120,2.28,0.700000,0.175877,1.306667,0.666638,0.187877,0.973333,0.033362,-0.011999,-0.877193,25.714286,23.024298,0.953333,0.213362,0.028495
8,N2,2.52,0.863636,0.246825,1.346667,0.381481,0.248545,1.173333,0.482155,-0.001719,-20.238095,-29.699248,13.077698,0.663333,0.225661,0.030560
9,hipr-1_ok1081,2.50,0.578947,0.219600,1.143333,0.508578,0.264993,1.356667,0.070369,-0.045393,4.800000,29.545455,44.607127,1.476667,0.241422,0.052565


In [26]:
# tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_XJ1']

In [174]:

def extract_phenotypes(df):
    ''' 
    input: one pd.DataFrame with multiple columns (1st column is the index, the other are phenotypes)
    returns: list of pd.DataFrame, each DataFrame with 2 columns: index and 1 phenotype, 
            for how many phenotypes there are in the input
    '''
    list_phenotypes_df = []
    phenotypes = df.columns[1:]
    index = df.columns[0]
    for i in phenotypes:
        new_df = df[[index, i]].copy()
        list_phenotypes_df.append(new_df)
    return list_phenotypes_df


# baseline_output_gene=baseline_output[baseline_output['dataset']==x]
# gene_baseline_data=baseline_output[baseline_output['Date'].isin(baseline_output_gene['Date'].unique())]
# gene_baseline_data_final=gene_baseline_data[gene_baseline_data['dataset'].isin(['N2', x])]

def calculate_MSD(list_of_dfs, by):
    '''
    input: list of dataframes with 2 columns (1st one is index, 2nd one are indiviudal phenotypes)
            output from extract_phenotypes function

    returns: the same list of dataframes with additional calculated columns for:
                    1. mean
                    2. count
                    3. std
                    4. ci95_hi
                    5. ci95_low
                    6. ci95
    '''
    new_list_of_dfs=[]
    for i in list_of_dfs:
        calculate_1= i.groupby(by=[by], as_index=False).agg(['mean', 'count', 'sem'])
        calculate_2= ci95(calculate_1)
        if Screen=="Neuron_Genes_Screen":
            N2_data = calculate_2[calculate_2.index=='N2'] if by=="Gene" else calculate_2[calculate_2.index.isin(['N2_XJ1','N2_N2'])]
        else:
            N2_data = calculate_2[calculate_2.index=='N2']
        calculate_2.iloc[:,0] = calculate_2.iloc[:,0]-N2_data.iloc[0,0]
        calculate_2.iloc[:,3] = calculate_2.iloc[:,3]-N2_data.iloc[0,0]
        calculate_2.iloc[:,4] = calculate_2.iloc[:,4]-N2_data.iloc[0,0]
        new_list_of_dfs.append(calculate_2)
    return new_list_of_dfs


def ci95(df):
    """
    input: df of 4 columns: index, mean, count, std

    returns: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

    """
    for metric in df.columns.levels[0]:
        if metric == 'Gene':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m, c, s = df[metric].loc[i]
                ci95_hi.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
    return df


def get_MSD(list_MSD):
    '''
    input: List of dataframes, each representing a phenotype with calculated MSD.

    returns: Single combined dataframe joining all input dataframes with MSD values.
    '''
    for a in list_MSD:
        if a.columns.levels[0] == list_MSD[0].columns.levels[0]:
            MSD=a
        else:
            MSD=MSD.join(a)
    return MSD



In [175]:
def get_combined_MSD(baseline_byplate,tap_byplate, by=['Gene','dataset']):
    """
    Combines MSD datafram from baseline plates and tap plates

    input:
        - baseline_byplate: baseline data by plate
        - tap_byplate: tap data by plate
        - by: what to group by "Gene" or "dataset"
    returns:
        - combined MSD dataframe
    """
    list_baseline_MSD=calculate_MSD(extract_phenotypes(baseline_byplate), by=by)

    list_tap_MSD=calculate_MSD(extract_phenotypes(tap_byplate), by=by)

    baseline_MSD = get_MSD(list_baseline_MSD)
    
    tap_MSD = get_MSD(list_tap_MSD)

    combined_MSD = baseline_MSD.join(tap_MSD, on=by)

    combined_MSD=combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                         "habit_prob": "Habituation of Respones Probability",
                                         "habit_speed":"Habituation of Response Speed",
                                         "init_dura": "Initial Response Duration",
                                         "init_prob": "Initial Response Probability",
                                         "init_speed": "Initial Response Speed",
                                         "final_dura": "Final Response Duration",
                                         "final_prob": "Final Response Probability",
                                         "final_speed": "Final Response Speed",
                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                         "recovery_speed": "Spontaneous Recovery of Response Speed",
                                         "memory_retention_dura": "Memory Retention of Response Duration",
                                         "memory_retention_prob": "Memory Retention of Response Probability",
                                         "memory_retention_speed": "Memory Retention of Response Speed"})

    combined_MSD=combined_MSD.reset_index()
    combined_MSD.columns = combined_MSD.columns.to_flat_index().str.join('-')
    combined_MSD=combined_MSD.rename(columns={by+"-": by})
    combined_MSD['Screen']=Screen
    
    return combined_MSD

## Gene-level SMD

In [176]:
combined_MSD=get_combined_MSD(baseline_output_byplate,
                              tap_data_byplate, 
                              by='Gene')

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrast

In [177]:
print(combined_MSD)

        Gene  Instantaneous Speed-mean  Instantaneous Speed-count  \
0    C03A3.3                 -0.052347                          4   
1    C03B1.5                 -0.016123                          4   
2    C04B4.2                 -0.085188                          4   
3    C04E6.5                 -0.031810                          4   
4    C27B7.7                 -0.019143                          4   
..       ...                       ...                        ...   
435    wwp-1                 -0.094905                          4   
436    xol-1                 -0.054133                          4   
437   zig-12                 -0.047364                          4   
438    zig-3                 -0.069833                          4   
439    zig-7                 -0.060913                          4   

     Instantaneous Speed-sem  Instantaneous Speed-ci95_hi  \
0                   0.005000                    -0.036435   
1                   0.006724                     

## Allele-level SMD

In [178]:
print(baseline_output_allele_byplate[baseline_output_allele_byplate.dataset=='N2_XJ1'])
print(tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_N2'])

     dataset  Instantaneous Speed  Interval Speed      Bias  Morphwidth  \
258   N2_XJ1             0.078686        0.081476  0.243865    0.107639   
262   N2_XJ1             0.135306        0.098450  0.628021    0.107281   
266   N2_XJ1             0.140936        0.097159  0.675668    0.104622   
270   N2_XJ1             0.150067        0.095419  0.723580    0.104706   
274   N2_XJ1             0.128076        0.095757  0.569462    0.096748   
...      ...                  ...             ...       ...         ...   
2556  N2_XJ1             0.136533        0.092684  0.766364    0.096246   
2557  N2_XJ1             0.144659        0.101192  0.699724    0.102023   
2558  N2_XJ1             0.096483        0.080709  0.569569    0.099812   
2559  N2_XJ1             0.124766        0.093121  0.666172    0.101687   
2560  N2_XJ1             0.090876        0.076118  0.525435    0.103336   

       Midline      Area  Angular Speed  Aspect Ratio       Kink      Curve  \
258   1.105549  0.14

In [179]:
allele_combined_MSD=get_combined_MSD(baseline_output_allele_byplate,
                                     tap_data_allele_byplate, 
                                     by='dataset')

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_distn_infrast

In [180]:
print(allele_combined_MSD)

            dataset  Instantaneous Speed-mean  Instantaneous Speed-count  \
0    C03A3.3_ok2834                 -0.049727                          4   
1    C03B1.5_ok2345                 -0.013503                          4   
2    C04B4.2_ok1212                 -0.082568                          4   
3    C04E6.5_ok1497                 -0.029190                          4   
4    C27B7.7_ok2868                 -0.016523                          4   
..              ...                       ...                        ...   
454    wwp-1_ok1102                 -0.092285                          4   
455        xol-1_y9                 -0.051513                          4   
456   zig-12_ok2994                 -0.044744                          4   
457    zig-3_ok1476                 -0.067213                          4   
458    zig-7_ok2329                 -0.058293                          4   

     Instantaneous Speed-sem  Instantaneous Speed-ci95_hi  \
0                   0.0050

## T-Stat analysis

In [181]:
def baseline_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for baseline analysis

    input:
        - by: what to group by "Gene" or "dataset"
    returns:
        - list_baseline_Tstats: to store t-statistics
        - list_baseline_metrics: to store metic names
    """
    PD_baseline_instantspeed_T=pd.DataFrame(columns = [by,"Instantaneous Speed"])
    PD_baseline_intspeed_T=pd.DataFrame(columns = [by,"Interval Speed"])
    PD_baseline_bias_T=pd.DataFrame(columns = [by,"Bias"])
    PD_baseline_morphwidth_T=pd.DataFrame(columns = [by,"Morphwidth"])
    PD_baseline_midline_T=pd.DataFrame(columns = [by,"Midline"])
    PD_baseline_area_T=pd.DataFrame(columns = [by,"Area"])
    PD_baseline_angularspeed_T=pd.DataFrame(columns = [by,"Angular Speed"])
    PD_baseline_aspectratio_T=pd.DataFrame(columns = [by,"Aspect Ratio"])
    PD_baseline_kink_T=pd.DataFrame(columns = [by,"Kink"])
    PD_baseline_curve_T=pd.DataFrame(columns = [by,"Curve"])
    PD_baseline_crab_T=pd.DataFrame(columns = [by,"Crab"])
    PD_baseline_pathlength_T=pd.DataFrame(columns = [by,"Pathlength"])

    list_baseline_Tstats=[PD_baseline_instantspeed_T,
                        PD_baseline_intspeed_T,
                        PD_baseline_bias_T,
                        PD_baseline_morphwidth_T,
                        PD_baseline_midline_T,
                        PD_baseline_area_T,
                        PD_baseline_angularspeed_T,
                        PD_baseline_aspectratio_T,
                        PD_baseline_kink_T,
                        PD_baseline_curve_T,
                        PD_baseline_crab_T,
                        PD_baseline_pathlength_T]

    list_baseline_metrics=["Instantaneous Speed",
                        "Interval Speed",
                        "Bias",
                        "Morphwidth",
                        "Midline",
                        "Area",
                        "Angular Speed",
                        "Aspect Ratio",
                        "Kink",
                        "Curve",
                        "Crab",
                        "Pathlength"]
    return list_baseline_Tstats, list_baseline_metrics

In [182]:
def tap_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for tap analysis

    input:
        - by: what to group by "Gene" or "dataset"
    returns:
        - list_baseline_Tstats: to store t-statistics
        - list_baseline_metrics: to store metic names
    """
    recovery_dura=pd.DataFrame(columns = [by,"Recovery Duration"])
    recovery_prob=pd.DataFrame(columns = [by,"Recovery Probability"])
    recovery_speed=pd.DataFrame(columns = [by,"Recovery Speed"])
    memory_retention_dura=pd.DataFrame(columns = [by,"Memory Retention Duration"])
    memory_retention_prob=pd.DataFrame(columns = [by,"Memory Retention Probability"])
    memory_retention_speed=pd.DataFrame(columns = [by,"Memory Retention Speed"])
    init_dura=pd.DataFrame(columns = [by,"Initial Duration"])
    init_prob=pd.DataFrame(columns = [by,"Initial Probability"])
    init_speed=pd.DataFrame(columns = [by,"Initial Speed"])
    final_dura=pd.DataFrame(columns = [by,"Final Duration"])
    final_prob=pd.DataFrame(columns = [by,"Final Probability"])
    final_speed=pd.DataFrame(columns = [by,"Final Speed"])
    hab_dura=pd.DataFrame(columns = [by,"Habituation of Duration"])
    hab_prob=pd.DataFrame(columns = [by,"Habituation of Probability"])
    hab_speed=pd.DataFrame(columns = [by,"Habituation of Speed"])

    list_tap_Tstats = [recovery_dura,
                    recovery_prob,
                    recovery_speed,
                    memory_retention_dura,
                    memory_retention_prob,
                    memory_retention_speed,
                    init_dura,
                    init_prob,
                    init_speed,
                    final_dura,
                    final_prob,
                    final_speed,
                    hab_dura,
                    hab_prob,
                    hab_speed]
    list_tap_metrics = ["recovery_dura",
                        "recovery_prob",
                        "recovery_speed",
                        "memory_retention_dura",
                        "memory_retention_prob",
                        "memory_retention_speed",
                        "init_dura",
                        "init_prob",
                        "init_speed",
                        "final_dura",
                        "final_prob",
                        "final_speed",
                        "habit_dura",
                        "habit_prob",
                        "habit_speed"]
    return list_tap_Tstats, list_tap_metrics

In [183]:
def TTest(Type, DF_ref, output, by=["Gene", "dataset"]):
    """
    Perform two sample t-test for each unique Gene/dataset column in the Df_ref
    input: 
        - a:column name of values 
        - DF_ref:reference dataframe
        - output: output df to store results in 
        - by: what to group by "Gene" or "dataset"
    """
    for a in DF_ref[by].unique():
        Tstat_a =ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele.isin(["XJ1","N2"])][Type],equal_var=False)[0]
        Tstat_g= ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        Tstat = Tstat_g if by=="Gene" else Tstat_a
        row=[a, Tstat]
        output.loc[len(output)]=row
    # print(output)

def do_TTest(by=["Gene", "dataset"], baseline=["true", "false"]):
    """
    Perform TTest function for each unique Gene/dataset column in baseline_output/tap_data
    
    input: 
        - by: what to group by "Gene" or "dataset"
        - baseline: whether or not to use baseline data

    returns: sorted T-statistics dataframe
    """

    if baseline=="true":
        list_Tstats,list_metrics=baseline_metrics(by)
        data = baseline_output
    else:
        list_Tstats,list_metrics=tap_metrics(by)
        data =tap_data
    for x in data[by].unique():
        if Screen=="Neuron_Genes_Screen":
            condition = x in (["N2"] if by == "Gene" else ["N2_XJ1", "N2_N2"])
        else:
            condition = (x =="N2")
        if condition:
            pass
        else:
            output_gene=data[data[by]==x]
            gene_data=data[data['Date'].isin(output_gene['Date'].unique())]
            if Screen=="Neuron_Genes_Screen":
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])] if by=="Gene" else gene_data[gene_data[by].isin(['N2_N2','N2_XJ1', x])]
            else:
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])]

            for a,b in zip(list_metrics, list_Tstats):
                TTest(a, gene_data_final, b, by) #calls t test function
    
    PD_Tstats=pd.DataFrame()
    for a in list_Tstats:
        b=a.groupby([by], as_index=False).mean()
        if b.columns.values[1] == list_Tstats[0].columns.values[1]:
            PD_Tstats=b
        else:
            PD_Tstats=PD_Tstats.join(b.iloc[:,1])
            
    PD_Tstats=PD_Tstats.set_index(by)
    return PD_Tstats
            

## T-stat analysis for Baseline data

In [184]:
print(baseline_output.head())

      Time   n  Number  Instantaneous Speed  Interval Speed   Bias  \
0  490.007  44      35               0.1804          0.1032  0.829   
1  490.040  44      35               0.1776          0.1044  0.829   
2  490.070  44      35               0.1732          0.1022  0.829   
3  490.107  44      35               0.1701          0.0984  0.857   
4  490.140  44      35               0.1756          0.0983  0.829   

   Morphwidth  Midline      Area  Angular Speed  ...  Kink  Curve    Crab  \
0      0.1087   1.0640  0.140010            5.6  ...  29.2   26.6  0.0131   
1      0.1091   1.0674  0.140572            5.3  ...  29.0   26.8  0.0139   
2      0.1095   1.0694  0.140718            5.2  ...  29.3   26.8  0.0132   
3      0.1099   1.0687  0.141072            5.1  ...  29.8   26.9  0.0131   
4      0.1101   1.0706  0.141468            5.1  ...  30.9   26.5  0.0147   

   Pathlength  Plate_id      Date               Screen dataset Gene Allele  
0      14.651   A0306aa  20100310  Neur

### Allele-level T-stat analysis of baseline data

In [185]:
PD_baseline_Tstats_allele = do_TTest("dataset", baseline="true") # get sorted T-statistics DataFrame 

In [186]:
PD_baseline_Tstats_allele

,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
dataset,,,,,,,,,,,,
C03A3.3_ok2834,-190.957166,-59.093280,-191.180979,-45.336975,-175.944216,-116.512954,-25.578709,74.654209,94.924872,92.264234,-42.391910,-8.012066
C03B1.5_ok2345,7.896495,52.814696,-25.494351,-3.372824,-44.948145,-17.373564,-3.274209,43.963327,23.827817,117.511393,11.048879,-98.373299
C04B4.2_ok1212,-802.684290,-225.302995,-581.538406,-211.390219,-930.813019,-682.003735,-154.303916,122.727550,155.567648,-54.324155,-321.010585,-153.185959
C04E6.5_ok1497,-222.837557,-56.077612,-147.002562,-226.912862,-325.288293,-356.825455,-53.760725,90.059869,109.589705,-13.590514,-204.869955,47.271796
C27B7.7_ok2868,-162.948777,17.108218,-168.026982,-607.866923,-1468.143139,-1295.500409,105.744052,202.041194,160.459022,55.737730,-13.479215,46.028674
...,...,...,...,...,...,...,...,...,...,...,...,...
wwp-1_ok1102,-844.053487,-308.320998,-559.510474,-534.351037,-1048.961175,-993.631155,-97.610932,195.974246,194.376830,23.593720,-358.533551,-55.949491
xol-1_y9,-264.748399,-157.835315,-222.700191,19.228221,26.174383,12.720984,-100.855157,70.230133,106.019590,112.897628,-148.048228,-36.496929
zig-12_ok2994,-252.951838,-117.113916,-283.533806,36.034517,-90.806844,-24.858545,-88.352275,89.434064,115.036622,55.519882,-107.532222,-88.079583


In [187]:
print(PD_baseline_Tstats_allele)
PD_baseline_Tstats_allele_sorted=PD_baseline_Tstats_allele.sort_index()


                Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
dataset                                                                       
C03A3.3_ok2834          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5_ok2345             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2_ok1212          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5_ok1497          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7_ok2868          -162.948777       17.108218 -168.026982 -607.866923   
...                             ...             ...         ...         ...   
wwp-1_ok1102            -844.053487     -308.320998 -559.510474 -534.351037   
xol-1_y9                -264.748399     -157.835315 -222.700191   19.228221   
zig-12_ok2994           -252.951838     -117.113916 -283.533806   36.034517   
zig-3_ok1476            -375.743600     -232.626497 -123.071956 -701.941777   
zig-7_ok2329            -258.933965      -99.697571 

### Gene-level T-stat analysis of baseline data

In [188]:
PD_baseline_Tstats=do_TTest("Gene", baseline="true") # get sorted T-statistics DataFrame 

In [189]:
PD_baseline_Tstats

,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
Gene,,,,,,,,,,,,
C03A3.3,-190.957166,-59.093280,-191.180979,-45.336975,-175.944216,-116.512954,-25.578709,74.654209,94.924872,92.264234,-42.391910,-8.012066
C03B1.5,7.896495,52.814696,-25.494351,-3.372824,-44.948145,-17.373564,-3.274209,43.963327,23.827817,117.511393,11.048879,-98.373299
C04B4.2,-802.684290,-225.302995,-581.538406,-211.390219,-930.813019,-682.003735,-154.303916,122.727550,155.567648,-54.324155,-321.010585,-153.185959
C04E6.5,-222.837557,-56.077612,-147.002562,-226.912862,-325.288293,-356.825455,-53.760725,90.059869,109.589705,-13.590514,-204.869955,47.271796
C27B7.7,-162.948777,17.108218,-168.026982,-607.866923,-1468.143139,-1295.500409,105.744052,202.041194,160.459022,55.737730,-13.479215,46.028674
...,...,...,...,...,...,...,...,...,...,...,...,...
wwp-1,-844.053487,-308.320998,-559.510474,-534.351037,-1048.961175,-993.631155,-97.610932,195.974246,194.376830,23.593720,-358.533551,-55.949491
xol-1,-264.748399,-157.835315,-222.700191,19.228221,26.174383,12.720984,-100.855157,70.230133,106.019590,112.897628,-148.048228,-36.496929
zig-12,-252.951838,-117.113916,-283.533806,36.034517,-90.806844,-24.858545,-88.352275,89.434064,115.036622,55.519882,-107.532222,-88.079583


In [190]:
print(PD_baseline_Tstats.head())
PD_baseline_Tstats_sorted=PD_baseline_Tstats.sort_index()


         Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
Gene                                                                   
C03A3.3          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7          -162.948777       17.108218 -168.026982 -607.866923   

             Midline         Area  Angular Speed  Aspect Ratio        Kink  \
Gene                                                                         
C03A3.3  -175.944216  -116.512954     -25.578709     74.654209   94.924872   
C03B1.5   -44.948145   -17.373564      -3.274209     43.963327   23.827817   
C04B4.2  -930.813019  -682.003735    -154.303916    122.727550  155.567648   
C04E6.5  -325.288293  -356.825455     -53.760725     90.059869  109.589705   
C27B7.7 -1468.143139 -1295.

## T-stat analysis for tap-response data

In [191]:
print(tap_data.head())

   init_dura  init_prob  init_speed  plate      Date Plate_id  \
0       1.93   0.954545    0.189637      1  20100516  C0512aa   
1       2.46   0.964286    0.203252      2  20100516  C0512ab   
2       2.47   0.954545    0.258704      4  20100516  C0512ad   
3       3.39   0.942857    0.316519      1  20100516  D0512aa   
4       3.51   0.971429    0.272080      2  20100516  D0512ab   

                Screen         dataset     Gene  Allele  ...  dist  \
0  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...   NaN   
1  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...   NaN   
2  Neuron_Genes_Screen  C03A3.3_ok2834  C03A3.3  ok2834  ...   NaN   
3  Neuron_Genes_Screen  C03B1.5_ok2345  C03B1.5  ok2345  ...   NaN   
4  Neuron_Genes_Screen  C03B1.5_ok2345  C03B1.5  ok2345  ...   NaN   

   recov_prob  recov_speed  taps  recovery_dura  recovery_prob  \
0         NaN          NaN   NaN            NaN            NaN   
1         NaN          NaN   NaN            NaN         

### Allele level T-stat analysis of tap response data

In [192]:
PD_habituation_Tstats_allele = do_TTest("dataset", baseline="false") # get sorted T-statistics DataFrame 

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, 

In [193]:
print(PD_habituation_Tstats_allele.head())
PD_habituation_Tstats_allele_sorted=PD_habituation_Tstats_allele.sort_index()


                   Recovery Duration  Recovery Probability  Recovery Speed  \
dataset                                                                      
C03A3.3_ok2834                   NaN                   NaN             NaN   
C03B1.5_ok2345                   NaN                   NaN             NaN   
C04B4.2_ok1212                   NaN                   NaN             NaN   
C04E6.5_ok1497                   NaN                   NaN             NaN   
C25H3.11_gk425293                NaN                   NaN             NaN   

                   Memory Retention Duration  Memory Retention Probability  \
dataset                                                                      
C03A3.3_ok2834                           NaN                           NaN   
C03B1.5_ok2345                           NaN                           NaN   
C04B4.2_ok1212                           NaN                           NaN   
C04E6.5_ok1497                           NaN                   

### Gene-level T-stat analysis of Tap response data

In [194]:
PD_habituation_Tstats = do_TTest("Gene", baseline="false") # get sorted T-statistics DataFrame 

/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/lavanya/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, 

In [195]:
print(PD_habituation_Tstats)
PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()


          Recovery Duration  Recovery Probability  Recovery Speed  \
Gene                                                                
C03A3.3                 NaN                   NaN             NaN   
C03B1.5                 NaN                   NaN             NaN   
C04B4.2                 NaN                   NaN             NaN   
C04E6.5                 NaN                   NaN             NaN   
C25H3.11           1.451037              0.161891        2.834033   
...                     ...                   ...             ...   
zig-12                  NaN                   NaN             NaN   
zig-3                   NaN                   NaN             NaN   
zig-7                   NaN                   NaN             NaN   
ztf-11                  NaN                   NaN             NaN   
ztf-6                   NaN                   NaN             NaN   

          Memory Retention Duration  Memory Retention Probability  \
Gene                             

# Merging t-stat data into one dataset

In [196]:
def pop_cols(combined):
    """
    Reorders columns in the combined dataframe. 
    (pops specific columns["Area","Midline", "Morphwidth","Angular Speed"] and
    reinserts at different positions)

    input:
        combined: dataframe with columns to be reordered

    """
    first_col=combined.pop("Area")
    combined.insert(0,"Area",first_col)

    first_col=combined.pop("Midline")
    combined.insert(0,"Midline",first_col)

    first_col=combined.pop("Morphwidth")
    combined.insert(0,"Morphwidth",first_col)

    first_col=combined.pop("Angular Speed")
    combined.insert(5,"Angular Speed",first_col)

def pop_last(combined):
    """
    Reorders the last three columns of the combined dataframe.
    input:
        combined: dataframe with columns to be reordered

    """
    last_col=combined.pop("Spontaneous Recovery of Response Duration")
    combined.insert(26,"Spontaneous Recovery of Response Duration",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Probability")
    combined.insert(26,"Spontaneous Recovery of Response Probability",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Speed")
    combined.insert(26,"Spontaneous Recovery of Response Speed",last_col)

    last_col=combined.pop("Memory Retention of Response Duration")
    combined.insert(26,"Memory Retention of Response Duration",last_col)

    last_col=combined.pop("Memory Retention of Response Probability")
    combined.insert(26,"Memory Retention of Response Probability",last_col)

    last_col=combined.pop("Memory Retention of Response Speed")
    combined.insert(26,"Memory Retention of Response Speed",last_col)

def rename_columns(df):
    '''
    Renames columns in the input dataframe
    input:
        combined: dataframe with columns to be renamed   
    returns:
        input dataframe with renamed columns 
    '''
    renames = {
        "Habituation of Duration": "Habituation of Response Duration",
        "Habituation of Probability": "Habituation of Respones Probability",
        "Habituation of Speed": "Habituation of Response Speed",
        "Initial Duration": "Initial Response Duration",
        "Initial Probability": "Initial Response Probability",
        "Initial Speed": "Initial Response Speed",
        "Final Duration": "Final Response Duration",
        "Final Probability": "Final Response Probability",
        "Final Speed": "Final Response Speed",
        "Recovery Duration": "Spontaneous Recovery of Response Duration",
        "Recovery Probability": "Spontaneous Recovery of Response Probability",
        "Recovery Speed": "Spontaneous Recovery of Response Speed",
        "Memory Retention Duration": "Memory Retention of Response Duration",
        "Memory Retention Probability": "Memory Retention of Response Probability",
        "Memory Retention Speed": "Memory Retention of Response Speed"
    }
    return df.rename(columns=renames)

def merge_Tstats(baseline, habituation, by=["Gene", "dataset"]):
    """
    merge two dataframes based on the Gene/dataset
    normalize the merged dataframe and then return it with melted version

    input:
        - baseline: baseline dataframe to merge
        - habituation: habituation dataframe to merge
        - by: what to group by "Gene" or "dataset"
    """

    #merge baseline and habituation data
    combined_Tstats = pd.merge(baseline, habituation, on=[by], how='left')
    combined_Tstats = combined_Tstats.sort_index() # sort by index
    print("combined_Tstats")
    print(combined_Tstats)

    #normalise combined dataframe by subtracting mean and div by sd
    combined_Tstats_normalized = (combined_Tstats-combined_Tstats.mean())/combined_Tstats.std()

    if by=="dataset" and Screen=="Neuron_Genes_Screen":
        print(combined_Tstats_normalized[combined_Tstats_normalized.index=="N2_XJ1"])
        combined_Tstats_normalized_2 = combined_Tstats_normalized-combined_Tstats_normalized[combined_Tstats_normalized.index=="N2_XJ1"].squeeze()
        print("combined Tstats normalized N2_XJ1 data")
        print(combined_Tstats_normalized_2[combined_Tstats_normalized_2.index=="N2_XJ1"])
    else :
        print(combined_Tstats_normalized[combined_Tstats_normalized.index=="N2"])
        combined_Tstats_normalized_2 = combined_Tstats_normalized-combined_Tstats_normalized[combined_Tstats_normalized.index=="N2"].squeeze()  
        print("combined Tstats normalized N2 data")
        print(combined_Tstats_normalized_2[combined_Tstats_normalized_2.index=="N2"])


    print("combined_Tstats_normalize")
    print(combined_Tstats_normalized)

    pop_cols(combined_Tstats) #reorder columns
    print("combined_Tstats")
    print(combined_Tstats)

    #rename columns of combined and normalized df
    combined_Tstats=rename_columns(combined_Tstats)
    combined_Tstats_normalized_2=rename_columns(combined_Tstats_normalized_2)

    print("combined_Tstats_normalize_2")
    print(combined_Tstats_normalized_2)
    
    pop_cols(combined_Tstats_normalized_2) #reorder columns
    pop_last(combined_Tstats_normalized_2) #reorder columns

    print("combined_Tstats_normalize_2")
    print(combined_Tstats_normalized_2)

    # Melt the combined dataframe
    combined_Tstats_melted=combined_Tstats.reset_index()
    combined_Tstats_melted=pd.melt(combined_Tstats_melted, id_vars=[by],
                                var_name='Metric',
                                value_name='T_score')
    # Sort the melted dataframe by T_score
    combined_Tstats_melted_sorted=combined_Tstats_melted.sort_values(by=['T_score'])
    print("combined_Tstats_melted_sorted")
    print(combined_Tstats_melted_sorted)

    # Melt the normalized dataframe
    combined_Tstats_normalized_melted=combined_Tstats_normalized_2.reset_index()
    combined_Tstats_normalized_melted=pd.melt(combined_Tstats_normalized_melted, id_vars=[by],
                                                   var_name='Metric',
                                                   value_name='T_score')
    print("combined_Tstats_normalized_melted")
    print(combined_Tstats_normalized_melted)

    #add Screen column to df and its melted version
    combined_Tstats_normalized_2['Screen']=Screen
    combined_Tstats_normalized_melted['Screen']=Screen

    return combined_Tstats_normalized_2,combined_Tstats_normalized_melted

## Gene-level

In [197]:
combined_Tstats_normalize_2, combined_Tstats_normalized_melted = merge_Tstats(PD_baseline_Tstats,PD_habituation_Tstats, "Gene")


combined_Tstats
         Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
Gene                                                                   
C03A3.3          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7          -162.948777       17.108218 -168.026982 -607.866923   
...                      ...             ...         ...         ...   
wwp-1            -844.053487     -308.320998 -559.510474 -534.351037   
xol-1            -264.748399     -157.835315 -222.700191   19.228221   
zig-12           -252.951838     -117.113916 -283.533806   36.034517   
zig-3            -375.743600     -232.626497 -123.071956 -701.941777   
zig-7            -258.933965      -99.697571 -215.421529  188.721797   

             Midline         Area  Angular Spee

## Allele level

In [198]:
combined_Tstats_normalize_allele_2, combined_Tstats_normalized_melted_allele = merge_Tstats(PD_baseline_Tstats_allele,PD_habituation_Tstats_allele, "dataset")

combined_Tstats
                Instantaneous Speed  Interval Speed        Bias  Morphwidth  \
dataset                                                                       
C03A3.3_ok2834          -190.957166      -59.093280 -191.180979  -45.336975   
C03B1.5_ok2345             7.896495       52.814696  -25.494351   -3.372824   
C04B4.2_ok1212          -802.684290     -225.302995 -581.538406 -211.390219   
C04E6.5_ok1497          -222.837557      -56.077612 -147.002562 -226.912862   
C27B7.7_ok2868          -162.948777       17.108218 -168.026982 -607.866923   
...                             ...             ...         ...         ...   
wwp-1_ok1102            -844.053487     -308.320998 -559.510474 -534.351037   
xol-1_y9                -264.748399     -157.835315 -222.700191   19.228221   
zig-12_ok2994           -252.951838     -117.113916 -283.533806   36.034517   
zig-3_ok1476            -375.743600     -232.626497 -123.071956 -701.941777   
zig-7_ok2329            -258.933965 

In [199]:
print(tap_output)

         time  dura   dist      prob     speed  plate      Date Plate_id  \
0     600.026  1.93  0.366  0.954545  0.189637      1  20100516  C0512aa   
1     610.094  1.93  0.395  0.857143  0.204663      1  20100516  C0512aa   
2     620.161  1.74  0.404  0.880000  0.232184      1  20100516  C0512aa   
3     630.194  1.44  0.303  0.966667  0.210417      1  20100516  C0512aa   
4     640.261  1.40  0.281  0.818182  0.200714      1  20100516  C0512aa   
..        ...   ...    ...       ...       ...    ...       ...      ...   
150   859.994  1.56  0.228  0.600000  0.146154      5  20160526  C0526ed   
151   869.960  1.45  0.237  0.250000  0.163448      5  20160526  C0526ed   
152   879.999  1.63  0.224  0.333333  0.137423      5  20160526  C0526ed   
153   889.958  3.43  0.656  0.333333  0.191254      5  20160526  C0526ed   
154  1189.993  5.40  0.902  0.142857  0.167037      5  20160526  C0526ed   

                  Screen  taps         dataset     Gene  Allele  
0    Neuron_Genes_Scr

In [200]:
print(combined_MSD)

        Gene  Instantaneous Speed-mean  Instantaneous Speed-count  \
0    C03A3.3                 -0.052347                          4   
1    C03B1.5                 -0.016123                          4   
2    C04B4.2                 -0.085188                          4   
3    C04E6.5                 -0.031810                          4   
4    C27B7.7                 -0.019143                          4   
..       ...                       ...                        ...   
435    wwp-1                 -0.094905                          4   
436    xol-1                 -0.054133                          4   
437   zig-12                 -0.047364                          4   
438    zig-3                 -0.069833                          4   
439    zig-7                 -0.060913                          4   

     Instantaneous Speed-sem  Instantaneous Speed-ci95_hi  \
0                   0.005000                    -0.036435   
1                   0.006724                     

## sqlite3 upload data

## A janky way to add data and update the sql 

1. Read table to pd.DataFrame
2. Add new data to pd.DataFrame
3. Replace old table with newly updated pd.DataFrame

In [201]:
# USE THIS CELL TO UPDATE ALL THE NEED TALBES (Also have baseline_output on the second line)

conn=sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/data_updated2.db')
tap_output.to_sql('tap_response_data', conn, if_exists='append', index=False)
baseline_output.to_sql('tap_baseline_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='append', index=False)
combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='append', index=False)
combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='append', index=False)
combined_MSD.to_sql('gene_MSD', conn, if_exists='append', index=False)
allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='append', index=False)
# combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)
print(conn.total_changes)
conn.close()


# Want to test edge cases of pd.to_sql functionality#############

7522633


## Done!